In [1]:
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install tiktoken

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import sklearn as sk
import sklearn.model_selection

import tiktoken

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def tokenizer(X):
    stop_words = set(stopwords.words('english'))
    num_tokens = 0
    num_sentences = 0
    tokenized_documents = []
    lemmatizer = WordNetLemmatizer()
    for text in X:
        sentences = sent_tokenize(text)
        num_sentences += len(sentences)
        tokenized_sentences = [nltk.RegexpTokenizer(r"\w+").tokenize(s) for s in sentences]
        tokenized_document = [lemmatizer.lemmatize(word.lower()) for sentence in tokenized_sentences for word in sentence if word.lower() not in stop_words]
        num_tokens += len(tokenized_document)
        tokenized_documents.append(tokenized_document)
    print("Number of tokens: ", num_tokens)
    print("Number of sentences: ", num_sentences)
    return tokenized_documents


import sklearn as sk

# tf-idf vectorizer
def vectorizer(tokenized_documents):
    ## tokenized_documents is a list of lists, where each inner list contains tokens
    vectorizer = sk.feature_extraction.text.TfidfVectorizer(lowercase=False, preprocessor=None, tokenizer=lambda x: x)
    X = vectorizer.fit_transform(tokenized_documents)
    return X, vectorizer

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [3]:
import sys
sys.path.append('../')
from tokenizer import tokenizer
from vectorizer import vectorizer

import numpy as np
import pandas as pd
import sklearn as sk

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [5]:
# FOR TESTING : only select first 20000 samples
# X, y = X[:20000], y[:20000]

In [6]:
tokenized_documents = tokenizer(X)

Number of tokens:  23767229
Number of sentences:  2832806


In [7]:
X, vect = vectorizer(tokenized_documents)

/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
# most frequent words
print("Top 10 most frequent words in the dataset")
print(vect.get_feature_names_out()[:10])

# least frequent words
print("Top 10 least frequent words in the dataset")
print(vect.get_feature_names_out()[-10:])

Top 10 most frequent words in the dataset
['0' '00' '000' '0000' '000001' '00001' '000013' '0000soo' '0001'
 '000111052']
Top 10 least frequent words in the dataset
['¾' 'â' 'çay' 'çaykur' 'çelem' 'être' 'île' 'ît' 'ø' 'þ']


In [9]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(454763, 111813) (113691, 111813) (454763,) (113691,)
  (0, 88023)	0.2703757583318995
  (0, 99992)	0.35728192737004544
  (0, 57108)	0.6842328554222189
  (0, 29768)	0.37700624873987243
  (0, 24559)	0.19202413658804746
  (0, 74441)	0.28581062226005205
  (0, 65294)	0.2300574892119257
  (0, 62655)	0.13210159908753327


# Feed Forward Neural Network
### Model starts here

In [11]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Add
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import datetime
import os

2024-05-04 00:26:13.713968: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-04 00:26:13.750203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 00:26:14.178197: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
def batch_generator(X, y, batch_size=32, num_classes=None):
    num_samples = X.shape[0]
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end].toarray()  # Convert only this slice to dense
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield (X_batch, y_batch)

# Setup the model
inputs = Input(shape=(X_train.shape[1],))
x = Dense(512, activation='relu')(inputs)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
residual = Dense(64, activation='relu')(x)  # Adjust the residual shape

x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Add()([x, residual])  # Add residual connection
x = Dense(32, activation='relu')(x)
outputs = Dense(np.max(y_train) + 1, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using a generator
train_generator = batch_generator(X_train, y_train, batch_size=32, num_classes=np.max(y_train) + 1)
model.fit(train_generator, epochs=20, steps_per_epoch=int(np.ceil(X_train.shape[0] / 32)))

Epoch 1/20


2024-05-04 00:26:14.875550: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78746 MB memory:  -> device: 0, name: NVIDIA H100 PCIe, pci bus id: 0000:b5:00.0, compute capability: 9.0
I0000 00:00:1714782376.787633   30388 service.cc:145] XLA service 0x7f38bc003890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714782376.787676   30388 service.cc:153]   StreamExecutor device (0): NVIDIA H100 PCIe, Compute Capability 9.0
2024-05-04 00:26:16.839417: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-04 00:26:17.095462: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


   16/14212 ━━━━━━━━━━━━━━━━━━━━ 2:31 11ms/step - accuracy: 0.4875 - loss: 1.7063

I0000 00:00:1714782381.482684   30388 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14212/14212 ━━━━━━━━━━━━━━━━━━━━ 177s 12ms/step - accuracy: 0.7201 - loss: 0.7768
Epoch 2/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 175s 12ms/step - accuracy: 0.8196 - loss: 0.5051
Epoch 3/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 178s 13ms/step - accuracy: 0.8860 - loss: 0.3269
Epoch 4/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 176s 12ms/step - accuracy: 0.9242 - loss: 0.2188
Epoch 5/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 189s 13ms/step - accuracy: 0.9482 - loss: 0.1515
Epoch 6/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 176s 12ms/step - accuracy: 0.9635 - loss: 0.1085
Epoch 7/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 170s 12ms/step - accuracy: 0.9736 - loss: 0.0804
Epoch 8/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 188s 13ms/step - accuracy: 0.9795 - loss: 0.0627
Epoch 9/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 174s 12ms/step - accuracy: 0.9835 - loss: 0.0510
Epoch 10/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 168s 12ms/step - accuracy: 0.9869 - loss: 0.0414
Epoch 11/20
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 183s 13ms/step - accuracy: 0.9890 -

In [13]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1)

In [14]:
# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
print("Confusion Matrix:")
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

print("\nClassification Report:")
print(metrics.classification_report(y_test, predictions))

3553/3553 ━━━━━━━━━━━━━━━━━━━━ 44s 11ms/step
Confusion Matrix:
[[ 7387   918   476   338  1207]
 [  769  3098   586   438   964]
 [  456   627  4466  1088  1848]
 [  242   279   897  8894  5811]
 [  501   451   800  3637 67513]]

Classification Report:
              precision    recall  f1-score   support

           1       0.79      0.72      0.75     10326
           2       0.58      0.53      0.55      5855
           3       0.62      0.53      0.57      8485
           4       0.62      0.55      0.58     16123
           5       0.87      0.93      0.90     72902

    accuracy                           0.80    113691
   macro avg       0.70      0.65      0.67    113691
weighted avg       0.79      0.80      0.80    113691



In [15]:
# # Save the model with current date and time in model folder

# Create a folder named _models in the current directory
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/feedforward_1_bis_feedforward_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

In [16]:
# test the model
test_sentences = [
    'This is a good product',
    'This is a bad product',
    'This is a product',
    'This is a very good product',
    'This is a very bad product',
    'That was bad'
]

for sentence in test_sentences:
    test_tokenized = tokenizer([sentence])
    test_vec = vect.transform(test_tokenized)
    test_vec_dense = test_vec.toarray()  # Convert sparse tensor to dense tensor
    test_vec_lstm = test_vec_dense.reshape(1, -1)
    result = model.predict(test_vec_lstm)
    predicted_class = result.argmax()
    predicted_score = predicted_class
    print(f"Test sentence: {sentence}")
    print(f"Predicted score: {predicted_score}")
    print()

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step
Test sentence: This is a good product
Predicted score: 5

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Test sentence: This is a bad product
Predicted score: 1

Number of tokens:  1
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Test sentence: This is a product
Predicted score: 1

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Test sentence: This is a very good product
Predicted score: 5

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Test sentence: This is a very bad product
Predicted score: 1

Number of tokens:  1
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Test sentence: That was bad
Predicted score: 1

